In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.insert(0, "..")

In [ ]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [ ]:
!ls ../pretrained/neural_fields/

In [ ]:
log_path = "../pretrained/neural_fields/jolteon"

In [ ]:
import yaml
import argparse
import os.path as osp
from train import dict2namespace
from pprint import pprint
from utils import load_imf

original, cfg = load_imf(
    log_path, 
    config_fpath=osp.join(log_path, "config.yaml"), 
    return_cfg=True)
original

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
import skimage
import skimage.measure
from trainers.utils.vis_utils import imf2mesh
    
original.eval()
res = 256
new_mesh = imf2mesh(original, res=res, threshold=0.00, normalize=True, norm_type='res')
new_mesh

In [ ]:
# verts = (new_mesh.vertices * 2 - res) / float(res)
# new_mesh = trimesh.Trimesh(vertices=verts, faces=new_mesh.faces)
print(new_mesh.vertices.max(), new_mesh.vertices.min())
mp.plot(new_mesh.vertices, new_mesh.faces)

# Creating Handles

1. Creating a set of deforming handles (i.e. sparse points on the surface that will change according to the shape)
2. Creating a set of statisc handles (i.e. sparse points on the surface that will remain unchanged)

In [ ]:
verts, faces = new_mesh.vertices, new_mesh.faces
print(verts.max(axis=0))
print(verts.min(axis=0))

In [ ]:
left_front = verts[(verts[:, 1] < -0.67) & (verts[:, 2] < 0) &  (verts[:, 0] < 0), :]
right_front = verts[(verts[:, 1] < -0.67) & (verts[:, 2] < 0) & (verts[:, 0] > 0), :]

right_back = verts[(verts[:, 1] < -0.67) & (verts[:, 2] > 0) & (verts[:, 0] > 0), :]
left_back = verts[(verts[:, 1] < -0.67) & (verts[:, 2] > 0) & (verts[:, 0] < 0), :]

p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(right_back, shading={"point_color": "blue", "point_size": 0.1})
p.add_points(left_back, shading={"point_color": "green", "point_size": 0.1})
p.add_points(right_front, shading={"point_color": "red", "point_size": 0.1})
p.add_points(left_front, shading={"point_color": "black", "point_size": 0.1})

In [ ]:
from scipy.spatial.transform import Rotation as ROT

def rot_and_translate(pts, rot_axis=None, rot_deg=None, translate=None):
    center = pts.mean(axis=0).reshape(1, 3)
    if rot_axis is None or rot_deg is None:
        rot_mat = np.eye(3)
    else:
        rot_mat = ROT.from_euler(rot_axis, rot_deg, degrees=True).as_matrix()
    if translate is None:
        translate = np.array([0, 0, 0])
    else:
        translate = np.array(translate).reshape(1, 3)
    transf = np.matmul(pts - center, rot_mat) + center + translate
    return transf

pts = left_front
# pts = nose

pts_out = rot_and_translate(pts, rot_axis='x', rot_deg=-30, translate=np.array([0, 0.3, -0.2]))
p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(pts, shading={'point_color': 'green', 'point_size': 0.1})
p.add_points(pts_out, shading={"point_color": "blue", "point_size": 0.1})

In [ ]:
handles, targets = [], []
for h, r_axis, r_angle, translate in [
    (left_front, 'x', -30, [0., 0.3, -0.2]),
    (right_front, 'x', -30, [0., 0.3, -0.2]),
    (left_back, None, None, [0., 0., 0.]),
    (right_back, None, None, [0., 0., 0.]),
]:
    handles.append(h.reshape(-1, 3))
    targets.append(rot_and_translate(h, rot_axis=r_axis, rot_deg=r_angle, translate=translate))
    
handles = np.concatenate(handles, axis=0)
targets = np.concatenate(targets, axis=0)

p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(handles, shading={"point_color": "blue", "point_size": 0.1})
p.add_points(targets, shading={"point_color": "red", "point_size": 0.1})

# Compare to Open3D ASAP Algorithm

In [ ]:
from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh
mesh_prime_tri = None
mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
                                 steps=100, smoothed_alpha=100)

In [ ]:
p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
p.add_points(handles, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(targets, shading={"point_color": "red", "point_size": 0.05})

In [ ]:
# from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh
# mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
#                                  steps=10000, smoothed_alpha=10)


In [ ]:
# p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
# p.add_points(handles, shading={"point_color": "blue", "point_size": 0.05})
# p.add_points(targets, shading={"point_color": "red", "point_size": 0.05})

# Save the Data for Neural Fields Deformation

In [ ]:
# # # Saving it to data
out_dir = "../data/jolteon/deform_jump.npy"
try:
    assert mesh_prime_tri
    np.save(out_dir, {
        "handles": handles,
        "targets": targets,
        'gtr_verts': mesh_prime_tri.vertices,
        'gtr_faces': mesh_prime_tri.faces
    })
except:
    print("Saving the handles and targets.")
    np.save(out_dir, {
        "handles": handles,
        "targets": targets,
    })

out_dir